In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.special as special 
from scipy.optimize import curve_fit 
import seaborn as sns 
from sklearn.metrics import r2_score 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [166]:
df = pd.read_csv('madrid_spain_Cleansed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26980 entries, 0 to 26979
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    26980 non-null  int64  
 1   id                            26980 non-null  int64  
 2   last_scraped                  26980 non-null  object 
 3   source                        26980 non-null  object 
 4   name                          26980 non-null  object 
 5   description                   26980 non-null  object 
 6   neighborhood_overview         26980 non-null  object 
 7   host_name                     26980 non-null  object 
 8   host_since                    26980 non-null  object 
 9   host_location                 26980 non-null  object 
 10  host_about                    26980 non-null  object 
 11  host_response_time            26980 non-null  object 
 12  host_response_rate            26980 non-null  object 
 13  h

In [167]:
#df["availability_365"] = pd.cut(df["availability_365"], 
#                                 bins=[-1, 182, 365], 
#                                 labels=["medioAñoOmenos", "masDeMedioAño"])

# df["host_total_listings_count"] = pd.cut(df["host_total_listings_count"], 
#                                 bins=[-1, 33, 67], 
#                                 labels=["De0-33", "De34-67"]) 

# df["host_verifications"] = df["host_verifications"].replace(["['email', 'phone', 'work_email']", "['email', 'phone']", "['email', 'work_email']", "['phone', 'work_email']"], "dosOMas")
# df["host_verifications"] = df["host_verifications"].replace(["['email']", "['phone']", '[]'], "ceroOUno")

df["room_type"] = df["room_type"].replace(["Entire home/apt", "Hotel room"], "EntireH&HotelRoom")
df["room_type"] = df["room_type"].replace(["Private room", "Shared room"], "Private&SharedRoom")


print(np.unique(df["room_type"]))


['EntireH&HotelRoom' 'Private&SharedRoom']


In [38]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
# dichotomic variables
varsDep = df["host_is_superhost"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

In [ ]:
from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [19]:
print(algoritmo.coef_)

[[ 0.14447741  0.10772879 -0.06023097]]


In [24]:
y_pred = np.where(algoritmo.predict_proba(X_test)[:,1] > 0.3, 't', 'f')
y_pred

print(np.unique(y_pred))

['f' 't']


In [25]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz")
print(matriz)

Matriz
[[5685  602]
 [1615  192]]


In [28]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print("Precision del modelo para f")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print("Precision del modelo para t")
print(precision)

Precision del modelo para f
0.7787671232876713
Precision del modelo para t
0.24181360201511334


In [29]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

Exactitud
0.7260934025203855


In [ ]:
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print("sensibilidad")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("sensibilidad")
print(sensibilidad)

sensibilidad
0.9042468585971052


# Segundo Modelo HostIdentityVerified

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_curve
from imblearn.over_sampling import SMOTE

# Variables independientes y dependientes
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
varsDep = df["host_identity_verified"]

# Creación de X e y
X = varsIndep
y = varsDep

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verificar balance de clases
print("Distribución original de y_train:")
print(y_train.value_counts())

# Aplicar SMOTE para balancear
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar balance después de SMOTE
print("Distribución después de SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Normalización
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Modelo
modelo = LogisticRegression()
modelo.fit(X_train_balanced, y_train_balanced)

# Encontrar el mejor umbral
probs = modelo.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs, pos_label="t")
best_threshold = thresholds[np.argmax(tpr - fpr)]

# Predicción con el mejor umbral
y_pred = np.where(probs > best_threshold, 't', 'f')

# Evaluación
print("Precisión para 'f':", precision_score(y_test, y_pred, average="binary", pos_label="f"))
print("Precisión para 't':", precision_score(y_test, y_pred, average="binary", pos_label="t"))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad para 'f':", recall_score(y_test, y_pred, average="binary", pos_label="f"))
print("Sensibilidad para 't':", recall_score(y_test, y_pred, average="binary", pos_label="t"))

Distribución original de y_train:
host_identity_verified
t    17064
f     1822
Name: count, dtype: int64
Distribución después de SMOTE:
host_identity_verified
t    17064
f     5119
Name: count, dtype: int64
Precisión para 'f': 0.16702127659574467
Precisión para 't': 0.9387561623056504
Exactitud: 0.6698789226587596
Sensibilidad para 'f': 0.593198992443325


# Tercer Modelo BEDS

In [87]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates"]]
# dichotomic variables
varsDep = df["beds"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("Precision del modelo igualOMayorDeDos")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menosDeDos")
print("Precision del modelo menosDeDos")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("sensibilidad igualOMayorDeDos")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menosDeDos")
print("sensibilidad menosDeDos")
print(sensibilidad)

['igualOMayorDeDos' 'menosDeDos']
Precision del modelo igualOMayorDeDos
0.6505952380952381
Precision del modelo menosDeDos
0.6444007858546169
Exactitud
0.6482579688658265
sensibilidad igualOMayorDeDos
0.7512027491408935
sensibilidad menosDeDos
0.5277554304102977


# Cuarto modelo  BEDROOMS

In [95]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds"]]
# dichotomic variables
varsDep = df["bedrooms"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("Precision del modelo igualOMayorDeDos")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menorDeDos")
print("Precision del modelo menosDeDos")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("sensibilidad igualOMayorDeDos")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menorDeDos")
print("sensibilidad menosDeDos")
print(sensibilidad)

['igualOMayorDeDos' 'menorDeDos']
Precision del modelo igualOMayorDeDos
0.8492856045475495
Precision del modelo menosDeDos
0.7533123028391168
Exactitud
0.830491722263405
sensibilidad igualOMayorDeDos
0.93394154417976
sensibilidad menosDeDos
0.5489655172413793


# Quinto modelo BATHROOMS

In [110]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["bathrooms"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="menosDeUnMedioBaño")
print("Precision del modelo menosDeUnMedioBaño")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="medioBaño")
print("Precision del modelo medioBaño")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menosDeUnMedioBaño")
print("sensibilidad menosDeUnMedioBaño")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="medioBaño")
print("sensibilidad menosDeDos")
print(sensibilidad)

['medioBaño' 'menosDeUnMedioBaño']
Precision del modelo menosDeUnMedioBaño
0.8054350968488002
Precision del modelo medioBaño
0.625
Exactitud
0.7792191746973066
sensibilidad menosDeUnMedioBaño
0.9266589057043073
sensibilidad menosDeDos
0.3531955790485344


# Sexto modelo instant bookable

In [ ]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["instant_bookable"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print("Precision del modelo t")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print("Precision del modelo f")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("sensibilidad t")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print("sensibilidad f")
print(sensibilidad)

['f' 't']
Precision del modelo menosDeUnMedioBaño
0.6263858093126385
Precision del modelo medioBaño
0.6613672496025437
Exactitud
0.6535705460835186
sensibilidad menosDeUnMedioBaño
0.34662576687116564
sensibilidad menosDeDos
0.8605709557302441


# septimo modelo Availability365

In [134]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["availability_365"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="masDeMedioAño")
print("Precision del modelo masDeMedioAño")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menosDeMedioAño")
print("Precision del modelo menosDeMedioAño")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="masDeMedioAño")
print("sensibilidad masDeMedioAño")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menosDeMedioAño")
print("sensibilidad menosDeMedioAño")
print(sensibilidad)

['masDeMedioAño' 'menosDeMedioAño']
Precision del modelo masDeMedioAño
0.5905779504613891
Precision del modelo menosDeMedioAño
0.6347970173985087
Exactitud
0.6235483073881888
sensibilidad masDeMedioAño
0.35555555555555557
sensibilidad menosDeMedioAño
0.8196405648267009


# Octavo modelo Host verifications

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_curve
from imblearn.over_sampling import SMOTE

# Variables independientes y dependientes
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
varsDep = df["host_verifications"]

# Creación de X e y
X = varsIndep
y = varsDep

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verificar balance de clases
print("Distribución original de y_train:")
print(y_train.value_counts())

# Aplicar SMOTE para balancear
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar balance después de SMOTE
print("Distribución después de SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Normalización
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Modelo
modelo = LogisticRegression()
modelo.fit(X_train_balanced, y_train_balanced)

# Encontrar el mejor umbral
probs = modelo.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs, pos_label="dosOMas")
best_threshold = thresholds[np.argmax(tpr - fpr)]

# Predicción con el mejor umbral
y_pred = np.where(probs > best_threshold, 'dosOMas', 'ceroOUno')

# Evaluación
print("Precisión para 'dosOMas':", precision_score(y_test, y_pred, average="binary", pos_label="dosOMas"))
print("Precisión para 'ceroOUno':", precision_score(y_test, y_pred, average="binary", pos_label="ceroOUno"))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad para 'dosOMas':", recall_score(y_test, y_pred, average="binary", pos_label="dosOMas"))
print("Sensibilidad para 'ceroOUno':", recall_score(y_test, y_pred, average="binary", pos_label="ceroOUno"))

Distribución original de y_train:
host_verifications
dosOMas     16913
ceroOUno     1973
Name: count, dtype: int64
Distribución después de SMOTE:
host_verifications
dosOMas     16913
ceroOUno     5073
Name: count, dtype: int64
Precisión para 'dosOMas': 0.9302381525138321
Precisión para 'ceroOUno': 0.1478282956565913
Exactitud: 0.5496664195700519
Sensibilidad para 'ceroOUno': 0.6674311926605505
Sensibilidad para 'dosOMas': 0.535447244530601


# Noveno modelo host_total_listings_count

In [163]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "host_listings_count", "bedrooms"]]
# dichotomic variables
varsDep = df["host_total_listings_count"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="De0-33")
print("Precision del modelo De0-33")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="De34-67")
print("Precision del modelo De34-67")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="De0-33")
print("sensibilidad De0-33")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="De34-67")
print("sensibilidad De34-67")
print(sensibilidad)

['De0-33' 'De34-67']
Precision del modelo De0-33
0.9719697939331883
Precision del modelo De34-67
0.7366548042704626
Exactitud
0.9638003459352606
sensibilidad De0-33
0.9903495044340115
sensibilidad De34-67
0.4859154929577465


# Decimo Modelo room_type

In [168]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "host_listings_count", "bedrooms"]]
# dichotomic variables
varsDep = df["room_type"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="EntireH&HotelRoom")
print("Precision del modelo EntireH&HotelRoom")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="Private&SharedRoom")
print("Precision del modelo Private&SharedRoom")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="EntireH&HotelRoom")
print("sensibilidad EntireH&HotelRoom")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Private&SharedRoom")
print("sensibilidad Private&SharedRoom")
print(sensibilidad)

['EntireH&HotelRoom' 'Private&SharedRoom']
Precision del modelo EntireH&HotelRoom
0.9027750442826216
Precision del modelo Private&SharedRoom
0.7487553932957186
Exactitud
0.8454410674573758
sensibilidad EntireH&HotelRoom
0.8583458083832335
sensibilidad Private&SharedRoom
0.8203636363636364
